In [ ]:
#!/usr/bin/env python

*********************************************************************<br>
MAIN PROGRAM TO COMPUTE LOVE NUMBERS (POTENTIAL/TIDE, LOAD, SHEAR, STRESS)<br>
<br>
Copyright (c) 2014-2019: HILARY R. MARTENS, LUIS RIVERA, MARK SIMONS         <br>
<br>
This file is part of LoadDef.<br>
<br>
   LoadDef is free software: you can redistribute it and/or modify<br>
   it under the terms of the GNU General Public License as published by<br>
   the Free Software Foundation, either version 3 of the License, or<br>
   any later version.<br>
<br>
   LoadDef is distributed in the hope that it will be useful,<br>
   but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
   GNU General Public License for more details.<br>
<br>
   You should have received a copy of the GNU General Public License<br>
   along with LoadDef.  If not, see <https://www.gnu.org/licenses/>.<br>
<br>
*********************************************************************

IMPORT MPI MODULE

In [ ]:
from mpi4py import MPI

MODIFY PYTHON PATH TO INCLUDE 'LoadDef' DIRECTORY

In [ ]:
import sys
import os
sys.path.append(os.getcwd() + "/../")

IMPORT PYTHON MODULES

In [ ]:
import numpy as np
from LOADGF.LN import compute_love_numbers 

--------------- SPECIFY USER INPUTS --------------------- #

In [ ]:
 
# Full path to planet model text file
#     Planet model should be spherically symmetric, elastic, 
#         non-rotating, and isotropic (SNREI)
#     Format: radius(km), vp(km/s), vs(km/s), density(g/cc)
#     If the file delimiter is not whitespace, then specify in
#         call to function. 
planet_model = ("../input/Planet_Models/PREM.txt")
 
# Extension for the output filename (Default is '.txt')
file_ext      = ("PREM.txt")

------------------ END USER INPUTS ----------------------- #

--------------------- SETUP MPI -------------------------- #

Get the main MPI communicator that controls communication between processors

In [ ]:
comm = MPI.COMM_WORLD
# Get my "rank", i.e. the processor number assigned to me
rank = comm.Get_rank()
# Get the total number of other processors used
size = comm.Get_size()

---------------------------------------------------------- #

-------------------- BEGIN CODE -------------------------- #

Ensure that the Output Directories Exist

In [ ]:
if (rank == 0):
    if not (os.path.isdir("../output/Love_Numbers/")):
        os.makedirs("../output/Love_Numbers/")
    if not (os.path.isdir("../output/Love_Numbers/LLN/")):
        os.makedirs("../output/Love_Numbers/LLN")
    if not (os.path.isdir("../output/Love_Numbers/PLN/")):
        os.makedirs("../output/Love_Numbers/PLN") 
    if not (os.path.isdir("../output/Love_Numbers/STR/")):
        os.makedirs("../output/Love_Numbers/STR")   
    if not (os.path.isdir("../output/Love_Numbers/SHR/")):
        os.makedirs("../output/Love_Numbers/SHR")

Make sure all jobs have finished before continuing

In [ ]:
comm.Barrier()

Compute the Love numbers (Load and Potential)

In [ ]:
if (rank == 0):

    # Compute Love Numbers
    ln_n,ln_h,ln_nl,ln_nk,ln_h_inf,ln_l_inf,ln_k_inf,ln_h_inf_p,ln_l_inf_p,ln_k_inf_p,\
        ln_hpot,ln_nlpot,ln_nkpot,ln_hstr,ln_nlstr,ln_nkstr,ln_hshr,ln_nlshr,ln_nkshr,\
        ln_planet_radius,ln_planet_mass,ln_sint,ln_Yload,ln_Ypot,ln_Ystr,ln_Yshr,\
        ln_lmda_surface,ln_mu_surface = \
        compute_love_numbers.main(planet_model,rank,comm,size,file_out=file_ext)
 
# For Worker Ranks, Run the Code But Don't Return Any Variables
else: 

    # Workers Compute Love Numbers
    compute_love_numbers.main(planet_model,rank,comm,size,file_out=file_ext)
 
    # Workers Will Know Nothing About the Data Used to Compute the GFs
    ln_n = ln_h = ln_nl = ln_nk = ln_h_inf = ln_l_inf = ln_k_inf = ln_h_inf_p = ln_l_inf_p = ln_k_inf_p = None
    ln_planet_radius = ln_planet_mass = ln_Yload = ln_Ypot = ln_Ystr = ln_Yshr = None
    ln_hpot = ln_nlpot = ln_nkpot = ln_hstr = ln_nlstr = ln_nkstr = ln_hshr = None
    ln_nlshr = ln_nkshr = ln_sint = ln_lmda_surface = ln_mu_surface = None 

--------------------- END CODE --------------------------- #